In [931]:
#STREAMS2TABLE
#written by Zachary J.J. Roper

#A script for converting raw event codes and timestamps into a trial-item dataset

In [932]:
#This notebook requires the following modules
import os
import numpy
import csv

In [933]:
#READ IN EVENT CODES
#The prepared file, out.csv, is a 2 column x session length (based on total number of trials) consistinting of event 
#codes (EVENTS[]) and their paired timestamps(TIMES[]). The file streams2text.m was used to generate these data 
#from the TDT data dump.

In [934]:
with open('out.csv') as csvfile:
    data = csv.reader(csvfile, delimiter = ',')
    events = []
    times = []
    for row in data:
        event = row[0]
        events.append(event)
        time = row[1]
        times.append(time)
pairedlist = list(zip(events,times))

In [935]:
events.count('2756')

8

In [936]:
trial = 0
triallist = [None]*len(pairedlist)
eclist = [None]*len(pairedlist)
for i, data in enumerate(pairedlist):
    if pairedlist[i][0] == '2301':
        trial += 1
    triallist[i] = trial  
    eclist[i] = i
uberlist = list(zip(triallist,eclist,pairedlist))

In [937]:
#uberlist

In [938]:
'1507' in uberlist[0][2]

True

In [939]:
RT = [None]*len(pairedlist)
for i, trial in enumerate(uberlist):
    #print(trial)
    if '2651' in uberlist[i][2]:
        RTbeg = float(uberlist[i][2][1])
        focusTrial = uberlist[i][0]
        for p, case in enumerate(uberlist):
            if '2810' in uberlist[p][2] and uberlist[i][2] == focusTrial:
                RTend = float(uberlist[p][2][1])
    RT[i] = RTend - RTbeg

In [940]:
#RT

In [941]:
#The following provides indexed lists for each of the individual event codes of interest 

In [942]:
TargetOnset = []
FixOnset = []
Saccade = []
for i, x in enumerate(events):
    #print(i,x)
    if x == '2301':
        FixOnset.append(i)
    if x == '2651':
        TargetOnset.append(i)
    if x == '2810':
        Saccade.append(i)

In [943]:
#float(times[74])-float(times[70])

In [944]:
#Saccade

In [945]:
#for i, x in enumerate(Saccade):
#    print(Saccade[i],TargetOnset[i])

In [946]:
TOtimes = []
for items in TargetOnset:
    TOtimes.append(times[items])

In [947]:
Trials = []
TimeTrials = []
for i, x in enumerate(FixOnset):
    if i < len(FixOnset)-1:
        a = FixOnset[i]
        b = FixOnset[i+1]-1
        Trials.append(events[a:b])
        TimeTrials.append(times[a:b])
    else:
        a = FixOnset[i]
        Trials.append(events[a:])
        TimeTrials.append(times[a:])

In [948]:
#print(TimeTrials[0])

In [949]:
path = os.getcwd()
csvfile = (path+"/table.csv")
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(Trials)

In [950]:
accuracy = [None]*len(Trials)
abort = [None]*len(Trials)
error = [None]*len(Trials)
catch = [None]*len(Trials)
correct = [None]*len(Trials)
for i, trial in enumerate(Trials):
    if '2600' in trial:
        accuracy[i] = 1 #Correct Saccade (2600)
    else:
        accuracy[i] = 0  
    if '2651' in trial: #fixation was maintained sufficiently long enough for target onset (2651)
        abort[i] = 0
    else:
        abort[i] = 1
    if '887' in trial:
        error[i] = 1
    else:
        error[i] = 0
    if '2756' in trial:
        catch[i] = 1
    else:
        catch[i] = 0
    if '888' in trial:
        correct[i] = 1
    else:
        correct[i] = 0

In [951]:
new_dict = {}
for i, trial in enumerate(Trials):
    new_dict[i+1] = dict(zip(Trials[i],TimeTrials[i]))

In [952]:
RT = [0]*len(Trials)
focusTrial = 0
for i, trial in enumerate(Trials):
    if '2651' in trial and '2810' in trial: #and '2600' in trial:
        focusTrial = i + 1
        for p, case in enumerate(uberlist):
            if uberlist[p][0] == focusTrial:
                if '2810' in uberlist[p][2][0]:
                    #print(uberlist[p][2][1])
                    RTend = float(uberlist[p][2][1])
                if '2651' in uberlist[p][2][0]:
                    #print(uberlist[p][2][1])
                    RTbeg = float(uberlist[p][2][1])
            RT[i] = RTend - RTbeg

In [953]:
#RT

In [954]:
checksum = sum(error)+sum(abort)+sum(catch)

In [955]:
checksum

281

In [956]:
path = os.getcwd()
csvfile = (path+"/behaviorArray.csv")
trial = 0
triallist = [None]*len(abort)
with open(csvfile, "w") as output:
    for i,x in enumerate(abort):
        trial += 1
        triallist[i] = trial 
        print(i+1,abort[i],accuracy[i],error[i],correct[i],catch[i],RT[i])
    header = ['Trial', 'Abort', 'Accuracy', 'Error', 'Correct', 'Catch', 'RT']    
    dataset = list(zip(triallist,abort,accuracy,error,correct,catch,RT))
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(dataset)

1 1 0 0 0 0 0
2 1 0 0 0 0 0
3 0 0 0 1 0 335.9950000000008
4 1 0 0 0 0 0
5 0 1 0 1 0 212.00890000000072
6 0 1 0 1 0 219.95520000000033
7 1 0 0 0 0 0
8 1 0 0 0 0 0
9 0 1 0 1 0 195.99369999999908
10 1 0 0 0 0 0
11 0 0 1 0 0 217.98919999999998
12 1 0 0 0 0 0
13 0 1 0 1 0 178.01209999999992
14 0 0 1 0 0 249.97899999999936
15 0 1 0 1 0 243.99869999999646
16 1 0 0 0 0 0
17 0 1 0 1 0 272.0153999999966
18 0 0 0 1 0 210.0020000000004
19 0 1 0 1 0 200.00770000000193
20 0 0 1 0 0 203.98070000000007
21 1 0 0 0 0 0
22 0 1 0 1 0 191.97960000000603
23 0 1 0 1 0 175.96409999999742
24 1 0 0 0 0 0
25 0 1 0 1 0 193.98669999999402
26 0 1 0 1 0 187.9652999999962
27 1 0 0 0 0 0
28 0 1 0 1 0 163.96279999999388
29 0 1 0 1 0 183.99229999999807
30 0 1 0 1 0 200.00769999998738
31 0 1 0 1 0 198.08270000001357
32 0 1 0 1 0 154.00959999999031
33 1 0 0 0 0 0
34 0 1 0 1 0 161.99680000000808
35 0 1 0 1 0 205.98799999999756
36 0 1 0 1 0 155.97580000000016
37 0 1 0 1 0 199.96689999999944
38 1 0 0 0 0 0
39 0 1 0 1 0 227.9

623 0 1 0 1 0 175.96409999998286
624 0 0 0 1 0 188.00639999983832
625 1 0 0 0 0 0
626 0 0 0 1 0 193.94570000004023
627 0 0 0 1 0 183.99230000004172
628 1 0 0 0 0 0
629 0 0 0 1 0 177.9710999999661
630 0 1 0 1 0 203.98090000008233
631 1 0 0 0 0 0
632 1 0 0 0 0 0
633 1 0 0 0 0 0
634 0 1 0 1 0 210.00190000003204
635 0 1 0 1 0 185.99940000008792
636 0 1 0 1 0 231.9564999998547
637 0 0 0 0 1 0
638 0 1 0 1 0 223.96919999993406
639 0 1 0 1 0 203.98090000008233
640 0 1 0 1 0 199.96669999998994
641 1 0 0 0 0 0
642 1 0 0 0 0 0
643 0 1 0 1 0 199.96669999998994
644 0 0 1 0 0 189.97239999985322
645 1 0 0 0 0 0
646 0 0 1 0 0 215.98199999984354
647 0 0 1 0 0 229.9495000001043
648 1 0 0 0 0 0
649 0 0 0 1 0 235.97050000005402
650 0 1 0 1 0 181.98529999982566
651 0 1 0 1 0 203.9806999999564
652 0 1 0 1 0 207.95390000008047
653 1 0 0 0 0 0
654 0 1 0 1 0 189.97239999985322
655 0 1 0 1 0 200.00769999995828
656 0 1 0 1 0 191.97939999983646
657 1 0 0 0 0 0
658 0 1 0 1 0 353.97629999998026
659 0 1 0 1 0 223.96

In [957]:
sum(catch)

8